In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

In [33]:
import pickle

In [3]:
df = pd.read_csv("diabetes.csv") # gerekli kütüphaneleri ekledikten sonra veri setini okutup df ismine atadık.

In [18]:
models = []


models.append(('GradientBoosting', GradientBoostingClassifier()))


In [19]:
df[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]] = df[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]].replace(0,np.NaN)

In [20]:
df.isnull().sum() # Her değişkenin kaç adet NaN değeri olduğunu gösteren kod bloğu

Pregnancies                   0
Glucose                      11
BloodPressure                44
SkinThickness                20
Insulin                     236
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
BMIRanges_Healthy             0
BMIRanges_Overweight          0
BMIRanges_Obese               0
INSULIN_DESC_Normal           0
NewGlucose_Normal             0
NewGlucose_Secret             0
NewGlucose_High               0
dtype: int64

In [21]:
df[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]].dropna()

,Glucose,BloodPressure,SkinThickness,Insulin,BMI
2,1.600000,-0.500,0.625,0.897033,-0.961538
3,-0.678788,-0.375,-0.625,-0.425185,-0.434066
4,0.484848,-2.000,0.875,0.441873,1.214286
6,-0.945455,-1.375,0.500,-0.495487,-0.115385
8,1.939394,-0.125,2.125,4.835744,-0.170330
...,...,...,...,...,...
759,1.769697,1.250,0.625,0.897033,0.379121
760,-0.703030,-0.875,-0.250,-1.339110,-0.401099
761,1.284848,0.125,0.375,0.897033,1.313187
763,-0.387879,0.250,2.500,0.582476,0.093407


In [22]:
naValues =["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

for i in naValues:
    df[i][(df[i].isnull()) & (df["Outcome"] == 0)] = df[i][(df[i].isnull()) & (df["Outcome"] == 0)].fillna(df[i][df["Outcome"] == 0].mean())
    df[i][(df[i].isnull()) & (df["Outcome"] == 1)] = df[i][(df[i].isnull()) & (df["Outcome"] == 1)].fillna(df[i][df["Outcome"] == 1].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist

In [23]:
for feature in df:

    Q1 = df[feature].quantile(0.05)
    Q3 = df[feature].quantile(0.95)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any( axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower) ].shape[0])
        print("lower",lower,"\nupper",upper)
        df.loc[df[feature] > upper,feature] = upper # baskılama
    else:
        print(feature, "no")

Pregnancies no
Glucose no
BloodPressure no
SkinThickness no
Insulin yes
3
lower -5.2115749655181824 
upper 6.177338959673344
BMI no
DiabetesPedigreeFunction no
Age no
Outcome no
BMIRanges_Healthy no
BMIRanges_Overweight no
BMIRanges_Obese no
INSULIN_DESC_Normal no
NewGlucose_Normal no
NewGlucose_Secret no
NewGlucose_High no


In [24]:
df['BMIRanges'] = pd.cut(x=df['BMI'], bins=[0,18.5,25,30,100],labels = ["Underweight","Healthy","Overweight","Obese"])

In [25]:
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"
df = df.assign(INSULIN_DESC=df.apply(set_insulin, axis=1)) #burada yukarıdaki fonksiyonu df'e uyguluyoruz.

In [26]:
df['NewGlucose'] = pd.cut(x=df['Glucose'], bins=[0,70,99,126,200],labels = ["Low","Normal","Secret","High"])

In [27]:
df = pd.get_dummies(df,drop_first=True)

In [28]:
from sklearn.preprocessing import RobustScaler
r_scaler = RobustScaler()

df_r = r_scaler.fit_transform(df.drop(["Outcome","BMIRanges_Healthy","BMIRanges_Overweight","BMIRanges_Obese","INSULIN_DESC_Normal","NewGlucose_Normal","NewGlucose_Secret","NewGlucose_High"],axis=1))

df_r = pd.DataFrame(df_r, columns=["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"])
df = pd.concat([df_r,df[["Outcome","BMIRanges_Healthy","BMIRanges_Overweight","BMIRanges_Obese","INSULIN_DESC_Normal","NewGlucose_Normal","NewGlucose_Secret","NewGlucose_High"]]],axis=1)

In [38]:
X = df.drop("Outcome",axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=46)

for name,model in models:
    mod = model.fit(X_train,y_train) #trainleri modele fit etmek
    y_pred = mod.predict(X_test) # tahmin
    acc = accuracy_score(y_test, y_pred) #rmse hesabı
    cvscore = cross_val_score(model, X,y, cv = 10).mean()
    print("Holdout Method:",end=" ")
    print(name,acc) #yazdırılacak kısım
    print("Cross Val Score",end=" ")
    print(name,cvscore)
    print("------------------------------------")

Holdout Method: GradientBoosting 0.8636363636363636
Cross Val Score GradientBoosting 0.8958817498291183
------------------------------------


In [34]:
filename = 'diabetes_model.pkl'
pickle.dump( GradientBoostingClassifier, open(filename, 'wb'))

In [35]:

model = open('diabetes_model.pkl','rb')
gradient = pickle.load(model)

In [40]:
y_pred = mod.predict(X_test)